In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/bad_images.zip'
!unzip '/content/drive/MyDrive/good_images.zip'
!unzip '/content/drive/MyDrive/all_images.zip'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from PIL import Image

In [2]:
# TRAIN_DATADIR = "/content"
TRAIN_DATADIR = r"C:\Users\lenovo\Desktop\gaip\Group-Project\Data Cleaning"

CATEGORIES = ["good_images", "bad_images"]
w_size = 128
h_size = 256
training_data = []

for category in CATEGORIES:
    path = os.path.join(TRAIN_DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img))
            img_array = cv2.resize(img_array, (w_size, h_size))
            training_data.append([img_array, class_num])
        except Exception as e:
            print(e)
            pass

In [3]:
random.shuffle(training_data)

In [4]:
len(training_data)

1273

In [5]:
x1 = []
y1 = []

for features, label in training_data:
    x1.append(features)
    y1.append(label)

x= np.array(x1)
x = x/255
y = np.array(y1)

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [7]:
xtrain.shape, xtest.shape

((1018, 256, 128, 3), (255, 256, 128, 3))

# Train new model
---

In [ ]:
# Model
Vgg = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False,input_shape=(256, 128,3))
Vgg.trainable = False

#Create new model on top
model=Sequential()
model.add(Vgg)
model.add(tf.keras.layers.Conv2D(256,(2,2),activation='relu'))
model.add(tf.keras.layers.Conv2D(516,(2,2),activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2),strides=(1, 1)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(2048, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 4, 512)         14714688  
                                                                 
 conv2d (Conv2D)             (None, 7, 3, 256)         524544    
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 2, 516)         528900    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 1, 516)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 5, 1, 516)         0         
                                                                 
 flatten (Flatten)           (None, 2580)              0         
                                                        

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=8)
model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)

In [ ]:
history = model.fit(x=xtrain, y=ytrain ,epochs = 50 , batch_size=32, validation_data=(xtest, ytest), callbacks=es)
# history = model.fit(x=xtrain, y=ytrain ,epochs = 50 , batch_size=32, validation_data=(xtest, ytest))

Epoch 1/50
35/35 [==============================] - 21s 203ms/step - loss: 0.6718 - accuracy: 0.5839 - val_loss: 0.5295 - val_accuracy: 0.7143
Epoch 2/50
35/35 [==============================] - 4s 125ms/step - loss: 0.4565 - accuracy: 0.7800 - val_loss: 0.4383 - val_accuracy: 0.7509
Epoch 3/50
35/35 [==============================] - 4s 124ms/step - loss: 0.3437 - accuracy: 0.8543 - val_loss: 0.2974 - val_accuracy: 0.8718
Epoch 4/50
35/35 [==============================] - 4s 125ms/step - loss: 0.2421 - accuracy: 0.8973 - val_loss: 0.4111 - val_accuracy: 0.8022
Epoch 5/50
35/35 [==============================] - 5s 134ms/step - loss: 0.2142 - accuracy: 0.9166 - val_loss: 0.4379 - val_accuracy: 0.8498
Epoch 6/50
35/35 [==============================] - 4s 129ms/step - loss: 0.1465 - accuracy: 0.9404 - val_loss: 0.3892 - val_accuracy: 0.8718
Epoch 7/50
35/35 [==============================] - 4s 127ms/step - loss: 0.0913 - accuracy: 0.9688 - val_loss: 0.3577 - val_accuracy: 0.8571
Epoch

In [ ]:
model.save('m2.h5')

# Load existing model
---

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [9]:
model = Model
model = load_model('m2.h5')

# Testing and checking model
---

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 4, 512)         14714688  
                                                                 
 conv2d (Conv2D)             (None, 7, 3, 256)         524544    
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 2, 516)         528900    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 1, 516)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 5, 1, 516)         0         
                                                                 
 flatten (Flatten)           (None, 2580)              0         
                                                        

In [ ]:
pred = model.predict(xtest)

In [ ]:
pred = np.array(pred)
pred.round()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, pred.round())

In [ ]:
for i in range(len(ytest)):
    if pred[i].round() == ytest[i] == 0:
        print('ok',pred[i],ytest[i])
        plt.imsave('/content/check/c'+str(i)+'.png',xtest[i])

    else:
        print('wrong')

# Classifying Data
---

In [ ]:
# TRAIN_DATADIR = "/content/all_images"
TRAIN_DATADIR = r"C:\Users\lenovo\Desktop\gaip\Group-Project\Data Cleaning\all_images"
flag=0
for img in os.listdir(TRAIN_DATADIR):
    img = cv2.imread(os.path.join(TRAIN_DATADIR, img))
    img = cv2.resize(img, (w_size,h_size))
    img = np.expand_dims(img, 0)
    result = model.predict(img)
    flag += 1
    print('image no.: ', flag)
    if result == 0:
        flag += 1
        #transfer to classified good_classified
        im = Image.fromarray(img[0])
        # im.save("/content/good_classified/" +str(i)+".png")
        im.save(r'C:/Users/lenovo/Desktop/gaip/Group-Project/Data Cleaning/classified_images/' + str(i) + '.png')
        # plt.imsave("/content/good_classified/" +str(i)+".png", img)
        print('good image')
    else:
        flag += 1
        #transfer to classified bad_classified
        # im = Image.fromarray(img[0])
        # im.save(r"C:\Users\lenovo\Desktop\gaip\Group-Project\Data Cleaning\bad_classified\" +str(i)+".png")
        # im.save("/content/bad_classified/" +str(i)+".png")
        # plt.imsave("/content/bad_classified/"+str(j)+ ".png", img)
        print('bad image')